In [10]:
import xlrd
from xlrd.sheet import ctype_text
import agate

# define types
text_type = agate.Text()
number_type = agate.Number()
boolean_type = agate.Boolean()
date_type = agate.Date()

In [31]:
# open workbooks
cpi_workbook = xlrd.open_workbook('corruption_perception_index.xls')
cpi_sheet = cpi_workbook.sheets()[0]

workbook = xlrd.open_workbook('unicef_oct_2014.xls')

In [33]:
sheet = workbook.sheets()[0]
title_rows = zip(sheet.row_values(4), sheet.row_values(5))
# combine the two title options into one string
titles = [t[0] + ' ' + t[1] for t in title_rows]

titles = [t.strip() for t in titles]
country_rows = [sheet.row_values(r) for r in range(6, 114)]

In [34]:
def remove_bad_chars(val):

# change '-' to null
    if val == '-':
        return None
    return val

cleaned_rows = []

# use remove_bad_chars to clean data
for row in country_rows:
    cleaned_row = [remove_bad_chars(rv) for rv in row]
    cleaned_rows.append(cleaned_row)

In [12]:
# function to identify types in a row
def get_types(example_row):
    types = []
    
    for v in example_row:
        value_type = ctype_text[v.ctype]
        if value_type == 'text':
            types.append(text_type)
        elif value_type == 'number':
            types.append(number_type)
        elif value_type == 'xldate':
            types.append(date_type)
        else: types.append(text_type)
        
    return types

In [13]:
# create table
def get_table(new_arr, types, titles):
    try:
        table = agate.Table(new_arr, titles, types)
        return table
    
    except Exception as e:
        print e

In [20]:
def get_new_array(old_array, function_to_clean):
    new_arr = []
    for row in old_array:
        cleaned_row = [function_to_clean(rv) for rv in row]
        new_arr.append(cleaned_row)
    return new_arr

In [22]:
def str_to_int(x):
    try:
        return int(x)
    except ValueError:
        print 'Could not convert: %s' % x
    return x

In [24]:
def float_to_str(x):
    try:
        return str(x)
    except ValueError:
        print 'Could not convert: %s' % x
    return x

In [36]:
example_row = sheet.row(6)
types = get_types(example_row)

In [37]:
table = agate.Table(cleaned_rows, titles, types)

C:\Users\ande5\Anaconda3\envs\py27\lib\site-packages\agate\utils.py:276: UnnamedColumnWarning: Column 2 has no name. Using "c".


In [38]:
table.print_table(max_columns=7)

| Countries and areas  | Total (%) | c | Sex (%) Male | Female | Place of residenc... | Rural | ... |
| -------------------- | --------- | - | ------------ | ------ | -------------------- | ----- | --- |
| Afghanistan          |      10.3 |   |         11.0 |    9.6 |                  7.9 |  10.8 | ... |
| Albania              |       5.1 | y |          6.2 |    3.9 |                      |       | ... |
| Algeria              |       4.7 | y |          5.5 |    3.9 |                  3.9 |   5.6 | ... |
| Angola               |      23.5 |   |         22.1 |   24.8 |                      |       | ... |
| Argentina            |       4.4 |   |          4.8 |    3.9 |                      |       | ... |
| Armenia              |       3.9 |   |          4.7 |    2.9 |                  2.4 |   6.0 | ... |
| Azerbaijan           |       6.5 | y |          7.5 |    5.4 |                  2.1 |  10.7 | ... |
| Bahrain              |       4.6 |   |          6.3 |    3.0 |                  

In [15]:
#identify types
cpi_types = get_types(cpi_sheet.row(3))
cpi_table = get_table(cpi_rows, cpi_types, cpi_titles)

C:\Users\ande5\Anaconda3\envs\py27\lib\site-packages\agate\utils.py:292: DuplicateColumnWarning: Column name "Country Rank" already exists in Table. Column will be renamed to "Country Rank_2".


In [17]:
# print title. We see that there are two Country Rank columns
print cpi_titles

[u'Country Rank', u'Country / Territory', u'WB Code', u'IFS Code', u'Region', u'Country Rank', u'CPI 2013 Score', u'Surveys Used', u'Standard Error', u'90% Confidence interval Lower', u'Upper', u'Scores range MIN', u'MAX', u'Data sources AFDB', u'BF (SGI)', u'BF (BTI)', u'IMD', u'ICRG', u'WB', u'WEF', u'WJP', u'EIU', u'GI', u'PERC', u'TI', u'FH']


In [18]:
#identify the duplicate column
cpi_titles[0] = cpi_titles[0] + ' Duplicate'
cpi_table = get_table(cpi_rows, cpi_types, cpi_titles)

In [25]:
# create final cpi table
cpi_rows = get_new_array(cpi_rows, float_to_str)
cpi_table = get_table(cpi_rows, cpi_types, cpi_titles)

Could not convert: Côte d´Ivoire


In [27]:
print cpi_titles

[u'Country Rank Duplicate', u'Country / Territory', u'WB Code', u'IFS Code', u'Region', u'Country Rank', u'CPI 2013 Score', u'Surveys Used', u'Standard Error', u'90% Confidence interval Lower', u'Upper', u'Scores range MIN', u'MAX', u'Data sources AFDB', u'BF (SGI)', u'BF (BTI)', u'IMD', u'ICRG', u'WB', u'WEF', u'WJP', u'EIU', u'GI', u'PERC', u'TI', u'FH']


In [39]:
#join tables
cpi_and_cl = cpi_table.join(table, 'Country / Territory', 'Countries and areas', inner = True)

In [41]:
cpi_and_cl.column_names

(u'Country Rank Duplicate',
 u'Country / Territory',
 u'WB Code',
 u'IFS Code',
 u'Region',
 u'Country Rank',
 u'CPI 2013 Score',
 u'Surveys Used',
 u'Standard Error',
 u'90% Confidence interval Lower',
 u'Upper',
 u'Scores range MIN',
 u'MAX',
 u'Data sources AFDB',
 u'BF (SGI)',
 u'BF (BTI)',
 u'IMD',
 u'ICRG',
 u'WB',
 u'WEF',
 u'WJP',
 u'EIU',
 u'GI',
 u'PERC',
 u'TI',
 u'FH',
 u'Total (%)',
 'c',
 u'Sex (%) Male',
 u'Female',
 u'Place of residence (%) Urban',
 u'Rural',
 u'Household wealth quintile (%) Poorest',
 u'Second',
 u'Middle',
 u'Fourth',
 u'Richest',
 u'Reference Year',
 u'Data Source')

In [42]:
# corruption score and % of child labor
for r in cpi_and_cl.order_by('CPI 2013 Score').limit(10).rows:
    print '{}: {} - {}%'.format(r['Country / Territory'], r['CPI 2013 Score'], r['Total (%)'])

Afghanistan: 8.0 - 10.3%
Somalia: 8.0 - 49.0%
Iraq: 16.0 - 4.7%
Yemen: 18.0 - 22.7%
Chad: 19.0 - 26.1%
Equatorial Guinea: 19.0 - 27.8%
Guinea-Bissau: 19.0 - 38.0%
Haiti: 19.0 - 24.4%
Cambodia: 20.0 - 18.3%
Burundi: 21.0 - 26.3%
